In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [2]:
df = pd.read_csv(r"C:\Users\desha\Data science\Gen_AI\equity_practice\for_practice_material\sample_text.csv")
df.shape

(8, 2)

In [3]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


In [4]:
import torch
print(torch.__version__)


2.2.1+cpu


In [5]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModel.from_pretrained("ProsusAI/finbert")

print("✅ FinBERT imported successfully!")

c:\Users\desha\.conda\envs\langchain_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\desha\.conda\envs\langchain_env\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\desha\.conda\envs\langchain_env\Lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\desha\.conda\envs\langchain_env\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please

✅ FinBERT imported successfully!


In [6]:
def mean_pooling(model_output, attention_mask):
    """Apply mean pooling on token embeddings."""
    token_embeddings = model_output.last_hidden_state  # (batch, seq_len, hidden_size)
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = input_mask_expanded.sum(1)
    return sum_embeddings / sum_mask  # (batch, hidden_size)

def encode(texts, batch_size=4):
    """Replicates SentenceTransformer.encode() using FinBERT."""
    all_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", truncation=True, padding=True, max_length=128)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings = mean_pooling(outputs, inputs["attention_mask"])
        all_embeddings.append(embeddings)
    return torch.cat(all_embeddings).cpu().numpy()

# Example usage
texts = df["text"].tolist()
vectors = encode(texts)

print(vectors.shape)  # (8, 768)
df["embedding"] = list(vectors)

(8, 768)


In [7]:
df.embedding

0    [0.32537353, 0.5706947, -0.44465637, 0.034282874, 0.26576868, -0.32804278, 1.0208964, 0.369243, ...
1    [0.231387, 0.5873888, -0.3331679, 0.27685255, 0.5955179, -0.31383783, -0.00960957, 0.9424848, -0...
2    [0.034566663, -0.13728066, 0.60293555, 0.002683533, 0.116821595, -0.8160711, 0.19304428, 1.31084...
3    [0.52608263, -0.16983034, 0.025238724, -0.07627583, 0.26528844, -0.6013145, 0.3495225, 0.5927164...
4    [0.04678474, -0.1000862, 0.75923496, -0.22278969, 0.17171718, -0.7896935, 0.29659304, 1.6121842,...
5    [0.19621563, -0.09797091, 0.04822123, -0.04966438, 0.56725454, -0.5944315, 0.100418046, 0.755917...
6    [0.24341223, -0.3020848, 0.36592636, 0.2119574, 0.83012307, -0.9028728, 0.29371384, 0.9738028, 0...
7    [0.21044843, -0.00919, 0.123780854, 0.33481956, 0.3446361, -0.5889844, -0.099441096, 0.4606435, ...
Name: embedding, dtype: object

In [8]:
df.embedding[0].shape

(768,)

In [9]:
df

,text,category,embedding
0,Meditation and yoga can improve mental health,Health,"[0.32537353, 0.5706947, -0.44465637, 0.034282874, 0.26576868, -0.32804278, 1.0208964, 0.369243, ..."
1,"Fruits, whole grains and vegetables helps control blood pressure",Health,"[0.231387, 0.5873888, -0.3331679, 0.27685255, 0.5955179, -0.31383783, -0.00960957, 0.9424848, -0..."
2,These are the latest fashion trends for this week,Fashion,"[0.034566663, -0.13728066, 0.60293555, 0.002683533, 0.116821595, -0.8160711, 0.19304428, 1.31084..."
3,Vibrant color jeans for male are becoming a trend,Fashion,"[0.52608263, -0.16983034, 0.025238724, -0.07627583, 0.26528844, -0.6013145, 0.3495225, 0.5927164..."
4,The concert starts at 7 PM tonight,Event,"[0.04678474, -0.1000862, 0.75923496, -0.22278969, 0.17171718, -0.7896935, 0.29659304, 1.6121842,..."
5,Navaratri dandiya program at Expo center in Mumbai this october,Event,"[0.19621563, -0.09797091, 0.04822123, -0.04966438, 0.56725454, -0.5944315, 0.100418046, 0.755917..."
6,Exciting vacation destinations for your next trip,Travel,"[0.24341223, -0.3020848, 0.36592636, 0.2119574, 0.83012307, -0.9028728, 0.29371384, 0.9738028, 0..."
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel,"[0.21044843, -0.00919, 0.123780854, 0.33481956, 0.3446361, -0.5889844, -0.099441096, 0.4606435, ..."


In [10]:
vectors

array([[ 0.32537353,  0.5706947 , -0.44465637, ..., -0.7846348 ,
        -0.7337736 , -0.22857611],
       [ 0.231387  ,  0.5873888 , -0.3331679 , ..., -0.722231  ,
        -0.67561394, -0.16845137],
       [ 0.03456666, -0.13728066,  0.60293555, ..., -0.830476  ,
        -0.22966653, -0.30232528],
       ...,
       [ 0.19621563, -0.09797091,  0.04822123, ..., -0.42305407,
        -0.21590513, -0.26118723],
       [ 0.24341223, -0.3020848 ,  0.36592636, ..., -0.5664116 ,
        -0.48368317, -0.5366415 ],
       [ 0.21044843, -0.00919   ,  0.12378085, ..., -0.49746177,
         0.00325318,  0.03749327]], dtype=float32)

In [11]:
vectors.shape

(8, 768)

In [12]:
dim = vectors.shape[1]
dim

768

In [13]:
import faiss

index = faiss.IndexFlatL2(dim)
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x00000174E4FD6EE0> >

In [14]:
index.add(vectors)

In [19]:
search_query = "looking for places to visit during holidays"

# Use your FinBERT encode() function
vec = encode([search_query])   # returns shape (1, 768)
print(vec.shape)               # (1, 768)

(1, 768)


In [20]:
distances, I = index.search(vec, k=2)

In [21]:
I

array([[6, 7]], dtype=int64)

In [22]:
df.iloc[I[0]]

,text,category,embedding
6,Exciting vacation destinations for your next trip,Travel,"[0.24341223, -0.3020848, 0.36592636, 0.2119574, 0.83012307, -0.9028728, 0.29371384, 0.9738028, 0..."
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel,"[0.21044843, -0.00919, 0.123780854, 0.33481956, 0.3446361, -0.5889844, -0.099441096, 0.4606435, ..."
